In [1]:
from syncode import Syncode
import warnings
warnings.filterwarnings('ignore')

model_name = "/data/share/models/hugging_face/Llama-7b"
grammar_file = open("invariants.lark", "r") 
grammar = grammar_file.read()
grammar_file.close()

# Load the Syncode augmented model
syn_llm = Syncode(model = model_name, mode='grammar_mask', grammar=grammar, parse_output_only=True)

/home/adharsh/miniconda3/envs/syncode/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.83it/s]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [2]:
program_text = """
#include <stdlib.h>
#define assume(e) if(!(e)) exit(-1);

int main() {
  
  int x;
  int y;
  
  (x = 1);
  (y = 0);
  
  while ((y < 1000)) {
    {
    (x  = (x + y));
    (y  = (y + 1));
    }

  }
  
{;
//@ assert( (x >= y) );
}

}
"""

In [3]:
output = syn_llm.infer(program_text)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [4]:
print(output)

['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']


# Syncode vs Standard LLM generation of Python program
We show that in following program the standard LLM generates a python program that has indentation error, while the Syncode LLM generates a correct python program.

In [16]:
partial_code = "def is_prime(n):\n    '''Return if prime'''\n  "
output = partial_code+llm.infer(partial_code)[0]
print(output)
exec(output)

def is_prime(n):
    '''Return if prime'''
   if n < 2:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True


IndentationError: unindent does not match any outer indentation level (<string>, line 3)

In [15]:
output = partial_code+syn_llm.infer(partial_code)[0]
print(output)
exec(output)

def is_prime(n):
    '''Return if prime'''
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
